In [654]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [655]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [656]:
stats = pd.read_csv(r'data/Seasons_Stats.csv', index_col=0)

In [657]:
stats_clean = stats.drop(['blanl', 'blank2', 'Tm'], axis=1)

In [658]:
stats_clean.head()

,Year,Player,Pos,Age,G,GS,MP,PER,TS%,3PAr,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,63.0,NaN,NaN,NaN,0.368,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,49.0,NaN,NaN,NaN,0.435,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,67.0,NaN,NaN,NaN,0.394,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,15.0,NaN,NaN,NaN,0.312,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,13.0,NaN,NaN,NaN,0.308,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [659]:
players = pd.read_csv(r'data/players.csv', index_col=0)
players.head(10)

,Player,height,weight,collage,born,birth_city,birth_state
0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky
5,Gene Berce,180.0,79.0,Marquette University,1926.0,NaN,NaN
6,Charlie Black,196.0,90.0,University of Kansas,1921.0,Arco,Idaho
7,Nelson Bobb,183.0,77.0,Temple University,1924.0,Philadelphia,Pennsylvania
8,Jake Bornheimer,196.0,90.0,Muhlenberg College,1927.0,New Brunswick,New Jersey
9,Vince Boryla,196.0,95.0,University of Denver,1927.0,East Chicago,Indiana


In [719]:
data = pd.merge(stats_clean, players[['Player', 'height', 'weight']], left_on='Player', right_on='Player', right_index=False,
      how='left', sort=False).fillna(value=0)
data = data[~(data['Pos']==0) & (data['MP'] > 200)]
data.reset_index(inplace=True, drop=True)
data['Player'] = data['Player'].str.replace('*','')

In [720]:
totals = ['PER', 'OWS', 'DWS', 'WS', 'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA',
         'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

for col in totals:
    data[col] = 36 * data[col] / data['MP']

In [721]:
totals = ['PER', 'OWS', 'DWS', 'WS', 'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA',
         'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

for col in totals:
    data[col] = 36 * data[col] / data['MP']

In [722]:
data.tail()

,Year,Player,Pos,Age,G,GS,MP,PER,TS%,3PAr,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,height,weight
19695,2017.0,Thaddeus Young,PF,28.0,74.0,74.0,2237.0,0.003859,0.562,0.172,...,0.082357,0.116284,0.031596,0.029524,0.007770,0.024862,0.034963,0.210813,203.0,100.0
19696,2017.0,Cody Zeller,PF,24.0,62.0,58.0,1725.0,0.007273,0.604,0.002,...,0.117595,0.176393,0.043118,0.027003,0.025261,0.028310,0.082317,0.278309,213.0,108.0
19697,2017.0,Tyler Zeller,C,27.0,51.0,5.0,525.0,0.061127,0.508,0.006,...,0.380865,0.583053,0.197486,0.032914,0.098743,0.094041,0.286824,0.836963,213.0,114.0
19698,2017.0,Paul Zipser,SF,22.0,44.0,18.0,843.0,0.012583,0.503,0.448,...,0.200605,0.227961,0.065653,0.027355,0.029179,0.072947,0.142247,0.437684,203.0,97.0
19699,2017.0,Ivica Zubac,C,19.0,38.0,11.0,609.0,0.059404,0.547,0.013,...,0.412337,0.555607,0.104831,0.048921,0.115315,0.104831,0.230629,0.992405,216.0,120.0


In [723]:
X = data.drop(['Player', 'Pos', 'G', 'GS', 'MP'], axis=1).as_matrix()
y = data['Pos'].as_matrix()

encoder = LabelBinarizer()
y_cat = encoder.fit_transform(y)
nlabels = len(encoder.classes_)

scaler =StandardScaler()
Xnorm = scaler.fit_transform(X)

stats2017 = (data['Year'] == 2017)
X_train = Xnorm[~stats2017]
y_train = y_cat[~stats2017]
X_test = Xnorm[stats2017]
y_test = y_cat[stats2017]

In [724]:
model = Sequential()
model.add(Dense(40, activation='relu', input_dim=46))
model.add(Dropout(0.5))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nlabels, activation='softmax'))

In [725]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [667]:
# x_train and y_train are Numpy arrays --just like in the Scikit-Learn API.
model.fit(X_train, y_train, epochs=200, batch_size=128, validation_split=0.2, verbose=1)

Train on 15378 samples, validate on 3845 samples
Epoch 1/200
15378/15378 [==============================] - 1s - loss: 2.8517 - acc: 0.1845 - val_loss: 2.0011 - val_acc: 0.4671
Epoch 2/200
15378/15378 [==============================] - 0s - loss: 2.1502 - acc: 0.3043 - val_loss: 1.4809 - val_acc: 0.4772
Epoch 3/200
15378/15378 [==============================] - 0s - loss: 1.8192 - acc: 0.3423 - val_loss: 1.2832 - val_acc: 0.4884
Epoch 4/200
15378/15378 [==============================] - 0s - loss: 1.6199 - acc: 0.3682 - val_loss: 1.1940 - val_acc: 0.5165
Epoch 5/200
15378/15378 [==============================] - 0s - loss: 1.5006 - acc: 0.3948 - val_loss: 1.1412 - val_acc: 0.5329
Epoch 6/200
15378/15378 [==============================] - 0s - loss: 1.4095 - acc: 0.4205 - val_loss: 1.1090 - val_acc: 0.5654
Epoch 7/200
15378/15378 [==============================] - 0s - loss: 1.3393 - acc: 0.4392 - val_loss: 1.0853 - val_acc: 0.5914
Epoch 8/200
15378/15378 [==============================

15378/15378 [==============================] - 0s - loss: 0.9702 - acc: 0.6079 - val_loss: 0.9302 - val_acc: 0.6700
Epoch 65/200
15378/15378 [==============================] - 0s - loss: 0.9755 - acc: 0.6024 - val_loss: 0.9451 - val_acc: 0.6978
Epoch 66/200
15378/15378 [==============================] - 0s - loss: 0.9727 - acc: 0.6024 - val_loss: 0.9325 - val_acc: 0.6947
Epoch 67/200
15378/15378 [==============================] - 0s - loss: 0.9635 - acc: 0.6157 - val_loss: 0.9247 - val_acc: 0.6796
Epoch 68/200
15378/15378 [==============================] - 0s - loss: 0.9622 - acc: 0.6085 - val_loss: 0.9303 - val_acc: 0.6871
Epoch 69/200
15378/15378 [==============================] - 0s - loss: 0.9645 - acc: 0.6124 - val_loss: 0.9312 - val_acc: 0.6882
Epoch 70/200
15378/15378 [==============================] - 0s - loss: 0.9647 - acc: 0.6144 - val_loss: 0.9230 - val_acc: 0.6692
Epoch 71/200
15378/15378 [==============================] - 0s - loss: 0.9743 - acc: 0.6085 - val_loss: 0.9265

15378/15378 [==============================] - 0s - loss: 0.9029 - acc: 0.6461 - val_loss: 0.8720 - val_acc: 0.6804
Epoch 128/200
15378/15378 [==============================] - 0s - loss: 0.9017 - acc: 0.6429 - val_loss: 0.8715 - val_acc: 0.6926
Epoch 129/200
15378/15378 [==============================] - 0s - loss: 0.9030 - acc: 0.6451 - val_loss: 0.8683 - val_acc: 0.6879
Epoch 130/200
15378/15378 [==============================] - 0s - loss: 0.9012 - acc: 0.6481 - val_loss: 0.8689 - val_acc: 0.6941
Epoch 131/200
15378/15378 [==============================] - 0s - loss: 0.8982 - acc: 0.6446 - val_loss: 0.8746 - val_acc: 0.6986
Epoch 132/200
15378/15378 [==============================] - 0s - loss: 0.8943 - acc: 0.6444 - val_loss: 0.8694 - val_acc: 0.6923
Epoch 133/200
15378/15378 [==============================] - 0s - loss: 0.9028 - acc: 0.6520 - val_loss: 0.8688 - val_acc: 0.6967
Epoch 134/200
15378/15378 [==============================] - 0s - loss: 0.8972 - acc: 0.6510 - val_loss:

15378/15378 [==============================] - 0s - loss: 0.8581 - acc: 0.6653 - val_loss: 0.8385 - val_acc: 0.6980
Epoch 191/200
15378/15378 [==============================] - 0s - loss: 0.8620 - acc: 0.6682 - val_loss: 0.8536 - val_acc: 0.6895
Epoch 192/200
15378/15378 [==============================] - 0s - loss: 0.8610 - acc: 0.6634 - val_loss: 0.8424 - val_acc: 0.6993
Epoch 193/200
15378/15378 [==============================] - 0s - loss: 0.8593 - acc: 0.6649 - val_loss: 0.8462 - val_acc: 0.6975
Epoch 194/200
15378/15378 [==============================] - 0s - loss: 0.8609 - acc: 0.6646 - val_loss: 0.8458 - val_acc: 0.6973
Epoch 195/200
15378/15378 [==============================] - 0s - loss: 0.8481 - acc: 0.6697 - val_loss: 0.8355 - val_acc: 0.6949
Epoch 196/200
15378/15378 [==============================] - 0s - loss: 0.8569 - acc: 0.6671 - val_loss: 0.8423 - val_acc: 0.6988
Epoch 197/200
15378/15378 [==============================] - 0s - loss: 0.8537 - acc: 0.6667 - val_loss:

In [669]:
model.test_on_batch(X_test, y_test, sample_weight=None)

[0.74056995, 0.68972749]

In [729]:
# Production model, using all data
model.fit(X_train, y_train, epochs=200, batch_size=128, validation_split=0, verbose=1)

Epoch 1/200
19223/19223 [==============================] - 1s - loss: 2.6888 - acc: 0.1762     
Epoch 2/200
19223/19223 [==============================] - 0s - loss: 1.9556 - acc: 0.3058     
Epoch 3/200
19223/19223 [==============================] - 0s - loss: 1.6586 - acc: 0.3508     
Epoch 4/200
19223/19223 [==============================] - 0s - loss: 1.5097 - acc: 0.3843     
Epoch 5/200
19223/19223 [==============================] - 0s - loss: 1.4272 - acc: 0.4068     
Epoch 6/200
19223/19223 [==============================] - 0s - loss: 1.3491 - acc: 0.4391     
Epoch 7/200
19223/19223 [==============================] - 0s - loss: 1.2954 - acc: 0.4582     
Epoch 8/200
19223/19223 [==============================] - 0s - loss: 1.2654 - acc: 0.4675     
Epoch 9/200
19223/19223 [==============================] - 0s - loss: 1.2277 - acc: 0.4859     
Epoch 10/200
19223/19223 [==============================] - 0s - loss: 1.2068 - acc: 0.5013     
Epoch 11/200
19223/19223 [=============

19223/19223 [==============================] - 0s - loss: 0.9364 - acc: 0.6238     
Epoch 86/200
19223/19223 [==============================] - 0s - loss: 0.9389 - acc: 0.6299     
Epoch 87/200
19223/19223 [==============================] - 0s - loss: 0.9364 - acc: 0.6196     
Epoch 88/200
19223/19223 [==============================] - 0s - loss: 0.9340 - acc: 0.6257     
Epoch 89/200
19223/19223 [==============================] - 0s - loss: 0.9408 - acc: 0.6235     
Epoch 90/200
19223/19223 [==============================] - 0s - loss: 0.9301 - acc: 0.6311     
Epoch 91/200
19223/19223 [==============================] - 0s - loss: 0.9218 - acc: 0.6324     
Epoch 92/200
19223/19223 [==============================] - 0s - loss: 0.9197 - acc: 0.6301     
Epoch 93/200
19223/19223 [==============================] - 0s - loss: 0.9235 - acc: 0.6292     
Epoch 94/200
19223/19223 [==============================] - 0s - loss: 0.9267 - acc: 0.6337     
Epoch 95/200
19223/19223 [=================

19223/19223 [==============================] - 0s - loss: 0.8647 - acc: 0.6634     
Epoch 169/200
19223/19223 [==============================] - 0s - loss: 0.8692 - acc: 0.6578     
Epoch 170/200
19223/19223 [==============================] - 0s - loss: 0.8722 - acc: 0.6614     
Epoch 171/200
19223/19223 [==============================] - 0s - loss: 0.8707 - acc: 0.6594     
Epoch 172/200
19223/19223 [==============================] - 0s - loss: 0.8659 - acc: 0.6579     
Epoch 173/200
19223/19223 [==============================] - 0s - loss: 0.8657 - acc: 0.6619     
Epoch 174/200
19223/19223 [==============================] - 0s - loss: 0.8699 - acc: 0.6586     
Epoch 175/200
19223/19223 [==============================] - 0s - loss: 0.8683 - acc: 0.6623     
Epoch 176/200
19223/19223 [==============================] - 0s - loss: 0.8661 - acc: 0.6623     
Epoch 177/200
19223/19223 [==============================] - 0s - loss: 0.8644 - acc: 0.6627     
Epoch 178/200
19223/19223 [=======

## Predicting the positions of the First NBA Team of 2017 ##

In [730]:
first_team_members = ['Russell Westbrook', 'James Harden', 'Anthony Davis', 'LeBron James', 'Kawhi Leonard']
first_team_stats = data[[((x[1]['Player'] in first_team_members) & (x[1]['Year']==2017)) for x in data.iterrows()]]
first_team_stats

,Year,Player,Pos,Age,G,GS,MP,PER,TS%,3PAr,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,height,weight
19326,2017.0,Anthony Davis,C,23.0,75.0,75.0,2708.0,0.004860,0.579,0.088,...,0.125831,0.156582,0.027746,0.016613,0.029514,0.031988,0.029690,0.370954,206.0,97.0
19399,2017.0,James Harden,PG,27.0,81.0,81.0,2947.0,0.004074,0.613,0.493,...,0.084164,0.098340,0.135199,0.017907,0.005521,0.069241,0.032084,0.351577,196.0,99.0
19444,2017.0,LeBron James,SF,32.0,74.0,74.0,2794.0,0.004482,0.619,0.254,...,0.090147,0.106251,0.107247,0.015274,0.007305,0.050303,0.022246,0.324397,203.0,113.0
19486,2017.0,Kawhi Leonard,SF,25.0,74.0,74.0,2474.0,0.005823,0.611,0.294,...,0.074109,0.091049,0.055053,0.027950,0.011646,0.032608,0.025832,0.399768,201.0,104.0
19671,2017.0,Russell Westbrook,PG,28.0,81.0,81.0,2802.0,0.005051,0.554,0.300,...,0.120006,0.142621,0.138659,0.021954,0.005117,0.072301,0.031363,0.422250,190.0,90.0


In [731]:
pd.DataFrame(index=first_team_stats.loc[:, 'Player'].values, data={'Real': first_team_stats.loc[:, 'Pos'].values,
    'Predicted':encoder.inverse_transform(model.predict(Xnorm[first_team_stats.index, :]))})

,Predicted,Real
Anthony Davis,PF,C
James Harden,PG,PG
LeBron James,SF,SF
Kawhi Leonard,SG,SF
Russell Westbrook,PG,PG


## Predicting the positions of the NBA MVP ##

In [732]:
mvp = [(1956, 'Bob Pettit'), (1957, 'Bob Cousy'), (1958, 'Bill Russell'), (1959, 'Bob Pettit'), 
(1960, 'Wilt Chamberlain'), (1961, 'Bill Russell'), (1962, 'Bill Russell'), (1963, 'Bill Russell'),
(1964, 'Oscar Robertson'), (1965, 'Bill Russell'), (1966, 'Wilt Chamberlain'), (1967, 'Wilt Chamberlain'),
(1968, 'Wilt Chamberlain'), (1969, 'Wes Unseld'), (1970, 'Willis Reed'), (1971, 'Lew Alcindor'), 
(1972, 'Kareem Abdul-Jabbar'), (1973, 'Dave Cowens'), (19704, 'Kareem Abdul-Jabbar'), (1975, 'Bob McAdoo'),
(1976, 'Kareem Abdul-Jabbar'), (1977, 'Kareem Abdul-Jabbar'), (1978, 'Bill Walton'), (1979, 'Moses Malone'), 
(1980, 'Kareem Abdul-Jabbar'), (1981, 'Julius Erving'), (1982, 'Moses Malone'), (1983, 'Moses Malone'), 
(1984, 'Larry Bird'), (1985, 'Larry Bird'), (1986, 'Larry Bird'), (1987, 'Magic Johnson'), 
(1988, 'Michael Jordan'), (1989, 'Magic Johnson'), (1990, 'Magic Johnson'), (1991, 'Michael Jordan'),
(1992, 'Michael Jordan'), (1993, 'Charles Barkley'), (1994, 'Hakeem Olajuwon'), (1995, 'David Robinson'),  
(1996, 'Michael Jordan'), (1997, 'Karl Malone'), (1998, 'Michael Jordan'), (1999, 'Karl Malone'), 
(2000, 'Shaquille O\'Neal'), (2001, 'Allen Iverson'), (2002, 'Tim Duncan'), (2003, 'Tim Duncan'), 
(2004, 'Kevin Garnett'), (2005, 'Steve Nash'), (2006, 'Steve Nash'), (2007, 'Dirk Nowitzki'), 
(2008, 'Kobe Bryant'), (2009, 'LeBron James'), (2010, 'LeBron James'), (2011, 'Derrick Rose'), 
(2012, 'LeBron James'), (2013, 'LeBron James'), (2014, 'Kevin Durant'), (2015, 'Stephen Curry'),
(2016, 'Stephen Curry')]

In [733]:
mvp_stats = pd.concat([data[(data['Player'] == x[1]) & (data['Year']==x[0])] for x in mvp], axis=0)

In [734]:
mvp_stats

,Year,Player,Pos,Age,G,GS,MP,PER,TS%,3PAr,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,height,weight
497,1956.0,Bob Pettit,C,23.0,72.0,0.0,2794.0,0.004532,0.502,0.000,...,0.000000,0.193244,0.031377,0.000000,0.000000,0.000000,0.033535,0.306965,206.0,92.0
536,1957.0,Bob Cousy,PG,28.0,64.0,0.0,2364.0,0.004870,0.452,0.000,...,0.000000,0.071659,0.110851,0.000000,0.000000,0.000000,0.031075,0.305883,185.0,79.0
688,1958.0,Bill Russell,C,23.0,69.0,0.0,2640.0,0.004240,0.465,0.000,...,0.000000,0.290826,0.037562,0.000000,0.000000,0.000000,0.033657,0.212355,208.0,97.0
784,1959.0,Bob Pettit,PF,26.0,72.0,0.0,2873.0,0.004428,0.519,0.000,...,0.000000,0.185589,0.034700,0.000000,0.000000,0.000000,0.031402,0.330511,206.0,92.0
827,1960.0,Wilt Chamberlain,C,23.0,72.0,0.0,3338.0,0.003257,0.493,0.000,...,0.000000,0.225766,0.019541,0.000000,0.000000,0.000000,0.017447,0.314862,216.0,124.0
992,1961.0,Bill Russell,C,26.0,78.0,0.0,3458.0,0.001962,0.454,0.000,...,0.000000,0.202457,0.029046,0.000000,0.000000,0.000000,0.016799,0.143280,208.0,97.0
1095,1962.0,Bill Russell,C,27.0,76.0,0.0,3433.0,0.002133,0.489,0.000,...,0.000000,0.196839,0.037498,0.000000,0.000000,0.000000,0.022763,0.157911,208.0,97.0
1208,1963.0,Bill Russell,C,28.0,78.0,0.0,3500.0,0.001925,0.464,0.000,...,0.000000,0.194982,0.036817,0.000000,0.000000,0.000000,0.019995,0.138487,208.0,97.0
1320,1964.0,Oscar Robertson,PG,25.0,79.0,0.0,3559.0,0.002824,0.576,0.000,...,0.000000,0.080114,0.088811,0.000000,0.000000,0.000000,0.028649,0.253747,196.0,92.0
1440,1965.0,Bill Russell,C,30.0,78.0,0.0,3466.0,0.002104,0.472,0.000,...,0.000000,0.202602,0.044232,0.000000,0.000000,0.000000,0.022008,0.118886,208.0,97.0


In [736]:
mvp_pred = pd.DataFrame(index=mvp_stats.loc[:, 'Player'].values, data={'Real': mvp_stats.loc[:, 'Pos'].values,
    'Predicted':encoder.inverse_transform(model.predict(Xnorm[mvp_stats.index, :]))})

In [737]:
mvp_pred

,Predicted,Real
Bob Pettit,C,C
Bob Cousy,PG,PG
Bill Russell,C,C
Bob Pettit,C,PF
Wilt Chamberlain,C,C
Bill Russell,C,C
Bill Russell,C,C
Bill Russell,C,C
Oscar Robertson,SG,PG
Bill Russell,C,C


# Does year metter? #

In [780]:
curry2017 = data[(data['Player'] == 'Stephen Curry') & (data['Year']==2017)] 
pettit1956 = data[(data['Player'] == 'Bob Pettit') & (data['Year']==1956)]

In [784]:
time_travel_curry = pd.concat([curry2017 for year in range(1956, 2018)], axis=0)
time_travel_curry['Year'] = range(1956, 2018)

X = time_travel_curry.drop(['Player', 'Pos', 'G', 'GS', 'MP'], axis=1).as_matrix()
y = time_travel_curry['Pos'].as_matrix()

y_cat = encoder.transform(y)
Xnorm = scaler.transform(X)

time_travel_curry_pred = pd.DataFrame(index=time_travel_curry.loc[:, 'Year'].values, 
                                data={'Real': time_travel_curry.loc[:, 'Pos'].values,
    'Predicted':encoder.inverse_transform(model.predict(Xnorm))})


time_travel_pettit = pd.concat([pettit1956 for year in range(1956, 2018)], axis=0)
time_travel_pettit['Year'] = range(1956, 2018)

X = time_travel_pettit.drop(['Player', 'Pos', 'G', 'GS', 'MP'], axis=1).as_matrix()
y = time_travel_pettit['Pos'].as_matrix()

y_cat = encoder.transform(y)
Xnorm = scaler.transform(X)

time_travel_pettit_pred = pd.DataFrame(index=time_travel_pettit.loc[:, 'Year'].values, 
                                data={'Real': time_travel_pettit.loc[:, 'Pos'].values,
    'Predicted':encoder.inverse_transform(model.predict(Xnorm))})

In [786]:
pd.concat([time_travel_curry_pred,time_travel_pettit_pred],axis=1,keys=['Stephen Curry','Bob Pettit'])

Stephen Curry      Bob Pettit     
         Predicted Real  Predicted Real
1956            SG   PG          C    C
1957            SG   PG          C    C
1958            SG   PG          C    C
1959            SG   PG          C    C
1960            SG   PG          C    C
1961            SG   PG          C    C
1962            SG   PG         PF    C
1963            SG   PG         PF    C
1964            SG   PG         PF    C
1965            SG   PG         PF    C
1966            SG   PG         PF    C
1967            SG   PG         PF    C
1968            SG   PG         PF    C
1969            SG   PG         PF    C
1970            SG   PG         PF    C
1971            SG   PG         PF    C
1972            SG   PG         PF    C
1973            SG   PG         PF    C
1974            SG   PG         PF    C
1975            SG   PG         PF    C
1976            SG   PG         PF    C
1977            SG   PG         PF    C
1978            SG   PG         PF    C
1979            SG   PG         PF    C
1980            SG   PG         PF    C
1981            PG   PG         PF    C
1982            PG   PG         PF    C
1983            PG   PG         PF    C
1984            PG   PG         PF    C
1985            PG   PG         PF    C
...            ...  ...        ...  ...
1988            PG   PG         PF    C
1989            PG   PG         PF    C
1990            PG   PG         PF    C
1991            PG   PG         PF    C
1992            PG   PG         PF    C
1993            PG   PG         PF    C
1994            PG   PG         PF    C
1995            PG   PG         SF    C
1996            PG   PG         SF    C
1997            PG   PG         SF    C
1998            PG   PG         SF    C
1999            PG   PG         SF    C
2000            PG   PG         SF    C
2001            PG   PG         SF    C
2002            PG   PG         SF    C
2003            PG   PG         SF    C
2004            PG   PG         SF    C
2005            PG   PG         SF    C
2006            PG   PG         SF    C
2007            PG   PG         SF    C
2008            PG   PG         SF    C
2009            PG   PG         SF    C
2010            PG   PG         SF    C
2011            PG   PG         SF    C
2012            PG   PG         SF    C
2013            PG   PG         SF    C
2014            PG   PG         SF    C
2015            PG   PG         SF    C
2016            PG   PG         SF    C
2017            PG   PG         SF    C

[62 rows x 4 columns]

# Does age metter? #

In [788]:
magic = data[(data['Player'] == 'Magic Johnson')] 
jordan = data[(data['Player'] == 'Michael Jordan')]

In [794]:
# Magic
X = magic.drop(['Player', 'Pos', 'G', 'GS', 'MP'], axis=1).as_matrix()
y = magic['Pos'].as_matrix()

y_cat = encoder.transform(y)
Xnorm = scaler.transform(X)

magic_pred = pd.DataFrame(index=magic.loc[:, 'Age'].values, 
                                data={'Real': magic.loc[:, 'Pos'].values,
    'Predicted':encoder.inverse_transform(model.predict(Xnorm))})

# Jordan
X = jordan.drop(['Player', 'Pos', 'G', 'GS', 'MP'], axis=1).as_matrix()
y = jordan['Pos'].as_matrix()

y_cat = encoder.transform(y)
Xnorm = scaler.transform(X)

jordan_pred = pd.DataFrame(index=jordan.loc[:, 'Age'].values, 
                                data={'Real': jordan.loc[:, 'Pos'].values,
    'Predicted':encoder.inverse_transform(model.predict(Xnorm))})

In [795]:
pd.concat([magic_pred,jordan_pred],axis=1,keys=['Magic Johnson','Michael Jordan'])

Magic Johnson      Michael Jordan     
         Predicted Real      Predicted Real
20.0            SG   SG            NaN  NaN
21.0            SG   SG             SG   SG
22.0            SF   SG             SG   SG
23.0            SG   SG             SG   SG
24.0            PG   PG             SG   SG
25.0            PG   PG             SG   SG
26.0            PG   PG             SG   SG
27.0            PG   PG             SG   SG
28.0            PG   PG             SG   SG
29.0            PG   PG             SG   SG
30.0            PG   PG            NaN  NaN
31.0            PG   PG             SG   SG
32.0           NaN  NaN             SG   SG
33.0           NaN  NaN             SG   SG
34.0           NaN  NaN             SG   SG
36.0            SF   PF            NaN  NaN
38.0           NaN  NaN             SG   SF
39.0           NaN  NaN             SF   SF

# How important are height and weight? #

In [798]:
first_team_stats

,Year,Player,Pos,Age,G,GS,MP,PER,TS%,3PAr,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,height,weight
19326,2017.0,Anthony Davis,C,23.0,75.0,75.0,2708.0,0.004860,0.579,0.088,...,0.125831,0.156582,0.027746,0.016613,0.029514,0.031988,0.029690,0.370954,206.0,97.0
19399,2017.0,James Harden,PG,27.0,81.0,81.0,2947.0,0.004074,0.613,0.493,...,0.084164,0.098340,0.135199,0.017907,0.005521,0.069241,0.032084,0.351577,196.0,99.0
19444,2017.0,LeBron James,SF,32.0,74.0,74.0,2794.0,0.004482,0.619,0.254,...,0.090147,0.106251,0.107247,0.015274,0.007305,0.050303,0.022246,0.324397,203.0,113.0
19486,2017.0,Kawhi Leonard,SF,25.0,74.0,74.0,2474.0,0.005823,0.611,0.294,...,0.074109,0.091049,0.055053,0.027950,0.011646,0.032608,0.025832,0.399768,201.0,104.0
19671,2017.0,Russell Westbrook,PG,28.0,81.0,81.0,2802.0,0.005051,0.554,0.300,...,0.120006,0.142621,0.138659,0.021954,0.005117,0.072301,0.031363,0.422250,190.0,90.0


In [824]:
multiplier = np.arange(0.8,1.2,0.02)
growing_predicted = []

for p in first_team_stats.iterrows():
    growing = pd.concat([p[1].to_frame().T for x in multiplier], axis=0)
    growing['height'] = growing['height'] * multiplier
    growing['weight'] = growing['weight'] * (multiplier ** 3)

    X = growing.drop(['Player', 'Pos', 'G', 'GS', 'MP'], axis=1).as_matrix()
    y = growing['Pos'].as_matrix()

    y_cat = encoder.transform(y)
    Xnorm = scaler.transform(X)

    growing_predicted.append(pd.DataFrame(index=multiplier, data={'height': growing.loc[:, 'height'].values,
            'Real': growing.loc[:, 'Pos'].values, 'Predicted':encoder.inverse_transform(model.predict(Xnorm))}))

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [825]:
pd.concat(growing_predicted,axis=1,keys=first_team_stats['Player'])

Player Anthony Davis              James Harden              LeBron James       \
           Predicted Real  height    Predicted Real  height    Predicted Real   
0.80              PF    C   164.8           PG   PG   156.8           PG   SF   
0.82              PF    C  168.92           PG   PG  160.72           PG   SF   
0.84              PF    C  173.04           PG   PG  164.64           PG   SF   
0.86              PF    C  177.16           PG   PG  168.56           PG   SF   
0.88              PF    C  181.28           PG   PG  172.48           PG   SF   
0.90              PF    C   185.4           PG   PG   176.4           PG   SF   
0.92              PF    C  189.52           PG   PG  180.32           PG   SF   
0.94              PF    C  193.64           PG   PG  184.24           PG   SF   
0.96              PF    C  197.76           PG   PG  188.16           SF   SF   
0.98              PF    C  201.88           PG   PG  192.08           SF   SF   
1.00              PF    C     206           PG   PG     196           SF   SF   
1.02              PF    C  210.12           PG   PG  199.92           PF   SF   
1.04               C    C  214.24           PG   PG  203.84           PF   SF   
1.06               C    C  218.36           SG   PG  207.76            C   SF   
1.08               C    C  222.48           SF   PG  211.68            C   SF   
1.10               C    C   226.6           PF   PG   215.6            C   SF   
1.12               C    C  230.72           PF   PG  219.52            C   SF   
1.14               C    C  234.84            C   PG  223.44            C   SF   
1.16               C    C  238.96            C   PG  227.36            C   SF   
1.18               C    C  243.08            C   PG  231.28            C   SF   

Player         Kawhi Leonard              Russell Westbrook              
        height     Predicted Real  height         Predicted Real height  
0.80     162.4            PG   SF   160.8                PG   PG    152  
0.82    166.46            PG   SF  164.82                PG   PG  155.8  
0.84    170.52            PG   SF  168.84                PG   PG  159.6  
0.86    174.58            PG   SF  172.86                PG   PG  163.4  
0.88    178.64            PG   SF  176.88                PG   PG  167.2  
0.90     182.7            SG   SF   180.9                PG   PG    171  
0.92    186.76            SG   SF  184.92                PG   PG  174.8  
0.94    190.82            SG   SF  188.94                PG   PG  178.6  
0.96    194.88            SG   SF  192.96                PG   PG  182.4  
0.98    198.94            SG   SF  196.98                PG   PG  186.2  
1.00       203            SG   SF     201                PG   PG    190  
1.02    207.06            SF   SF  205.02                PG   PG  193.8  
1.04    211.12            SF   SF  209.04                SG   PG  197.6  
1.06    215.18            PF   SF  213.06                SF   PG  201.4  
1.08    219.24            PF   SF  217.08                SF   PG  205.2  
1.10     223.3             C   SF   221.1                SF   PG    209  
1.12    227.36             C   SF  225.12                PF   PG  212.8  
1.14    231.42             C   SF  229.14                PF   PG  216.6  
1.16    235.48             C   SF  233.16                 C   PG  220.4  
1.18    239.54             C   SF  237.18                 C   PG  224.2